# From S3 to Postgres

### Introduction

In this lesson, we'll use redshift to store data that originally came from the `foursquare_api` application about our venues zipcodes.  We currently have the data in CSV form, and we'll load this CSV file from to S3, and then import the data over to redshift.  Let's get started.

### Connecting to Postgres

Let's get started by connecting to our redshift database.  Because redshift is derived from postgres, we can use our `psycopg2` library to connect to the database.  To do so, we'll need to provide our redshift endpoint as the host.  We can find this on the redshift dashboard.

> <img src="./redshift-cluster.png" width="80%">

We'll also need to specify the user, password, and database name that we specified when creating our redshift cluster.

In [179]:
import psycopg2
endpoint = "redshift-cluster-1.cdpgnoufdsdf.us-east-1.redshift.amazonaws.com"

conn = psycopg2.connect(
    host=endpoint,
    database="dev",
    port = "5439",
    user="awsuser",
    password="Password1")

In [180]:
cursor = conn.cursor()

Now that we've connected to the database, the next step is to create a table in redshift and load in some data from a csv file stored in S3.  

So let's create the table our zipcodes table in redshift.  Below is the command to do so.

In [181]:
create_zipcodes_query = """CREATE TABLE "zipcodes" (
    "id" integer NOT NULL DEFAULT nextval('zipcodes_id_seq'),
    "code" INTEGER,
    "city_id" INTEGER
);"""

In [182]:
cursor.execute(create_zipcodes_query)

In [183]:
conn.commit()

### Loading Data to S3

For now, we can load in data to S3 simply by uploading it through the AWS console.  We can visit the S3 dashboard [here](https://s3.console.aws.amazon.com/s3).  

> <img src="./s3-dashboard.png" width="70%">

We can add in some data by creating a new bucket.  And then simply dragging and dropping a csv file.

<img src="./s3-data.png" width="60%">

### Adding Some Data

Now that our data is in S3, we can copy it from S3 into AWS redshift.  Here's how we do so.

In [171]:
qry = """COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
region 'us-east-1';
"""

cursor.execute(qry)

In [172]:
conn.commit()

In [173]:
cursor.execute('SELECT * FROM zipcodes LIMIT 2;')
cursor.fetchall()

[(47, 19019, 52), (48, 10001, 50)]

Ok, so let's break down thee command above.

```sql
COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
```

The command takes the form `COPY table_name (table columns) from s3://csv_file.csv`.  Then in the next line we specify the credentials that grants us access to the S3 file.  Notice that we have a key of the `aws_iam_role` and that the role we are using is the `redshiftRole`.  We get the Role ARN by going to that role's dashboard.

> <img src="./redshift-role.png" width="70%">

Then finally, we provided optional parameters of the delimeter and IGNOREHEADER 1, and finished with the redshift region.

> Take another look at the query and see if it makes more sense.

In [ ]:
qry = """COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
region 'us-east-1';
"""

### Debugging Copy Commands

Unfortunately, redshift is fairly strict in terms of it's copy commands.  To debug redshift, there are a couple of things that we can do.

The first, is to attempt the query directly through the dashboard.

> <img src="./query_editor.png" width="80%">

The next is to take a look at what went wrong by reading the error message.  


Let's see this by way of example.  We'll provide an invalid query by removing the `IGNOREHEADER 1` flag.  This time around, redshift will try to read in the first line of the csv as values, and will see that the first row of `id, code, city_id` cannot be inserted into the zipcode table as integers.

In [184]:
qry = """COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
region 'us-east-1';
"""

cursor.execute(qry)

InternalError_: Load into table 'zipcodes' failed.  Check 'stl_load_errors' system table for details.


Unfortunately, the error message is not directly apparent.  We see an error message of the following:
```python
Load into table 'zipcodes' failed.  Check 'stl_load_errors' system table for details.
```

To see what went wrong we'll have to query the `stl_load_errors` table to view the most recent errors.  Below is the query to view the error.

In [168]:
qry = """select query, substring(filename,22,25) as filename,line_number as line, 
substring(colname,0,12) as column, type, position as pos, substring(raw_line,0,30) as line_text,
substring(raw_field_value,0,15) as field_text, 
substring(err_reason,0,45) as reason
from stl_load_errors 
order by query desc
limit 1;"""

In [169]:
cursor.execute(qry)

In [170]:
cursor.fetchall()

# [(1041,
#   'ta/zipcodes.csv',
#   1,
#   'id',
#   'int4      ',
#   0,
#   'id,code,city_id',
#   'id',
#   "Invalid digit, Value 'i', Pos 0, Type: Integ")]

[(1041,
  'ta/zipcodes.csv',
  1,
  'id',
  'int4      ',
  0,
  'id,code,city_id',
  'id',
  "Invalid digit, Value 'i', Pos 0, Type: Integ")]

And doing so, we can see the query number, the file that was read, the line number of the error (line 1), and in the last column, the reason.  Here, we had an our CSV file contained a value of `i` (from the id) when it needed an Integer.  So we can tell redshift to skip the header line, so that the values it reads in are the correct type.

### Summary

In this lesson, we saw how to read data into our redshift database from S3.  We did so by first connecting to our database using `psycopg2` with a connecting to the redshift endpoint.  We then created our table in redshift.  Next was to copy over our data from S3 and into redshift, which we did with the following command.

In [ ]:
qry = """COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
region 'us-east-1';
"""

Finally, we saw that to debug our copy command in redshift we needed to `stl_load_errors` table to view a more detailed error message.

### Resources

[Loading Data From s3](https://docs.aws.amazon.com/redshift/latest/dg/tutorial-loading-data.html)

[S3 to Redshift](https://www.sqlshack.com/load-data-into-aws-redshift-from-aws-s3/)